# Deep Q Network (DQN)

Reference Paper: [Playing Atari with Deep Reinforcement Learning](https://arxiv.org/abs/1312.5602)

## DQN

### Experience

time step $t$에서 $t+1$로 transition 발생 시 관찰된 경험:

* `obs`: `(num_envs, *obs_shape)`
* `action`: `(num_envs, 1)`
* `next_obs`: `(num_envs, *obs_shape)`
* `reward`: `(num_envs, 1)`
* `terminated`: `(num_envs, 1)`

`obs`는 observation으로 state와 대응되는 개념임. 다만 일반적인 환경에서 state에 대한 모든 정보가 아닌 관찰된 일부분만 얻기 때문에 보통 observation이라고 명칭함.

In [14]:
from typing import NamedTuple
import numpy as np

class Experience(NamedTuple):
    obs: np.ndarray
    action: np.ndarray
    next_obs: np.ndarray
    reward: np.ndarray
    terminated: np.ndarray

### Replay Buffer

경험을 replay buffer에 저장해 놓았다가 training 시 경험을 랜덤하게 샘플링한다.

In [15]:
from typing import Optional, Tuple
import operator
import torch

class ReplayBuffer:
    def __init__(self, 
                 training_freq: int,
                 sampled_batch_size: int,
                 capacity: int,
                 num_envs: int) -> None:
        self.training_freq = training_freq
        self.sampled_batch_size = sampled_batch_size
        self.capacity = capacity
        self.num_envs = num_envs
        self.reset()
        
    def reset(self):
        """Reset experience replay, which is to clear memories."""
        self.n_step = 0
        self.count = 0
        self.recent_idx = -1
        
        self.obs = [None] * self.capacity
        self.action = [None] * self.capacity
        self.next_obs = [None] * self.capacity
        self.reward = [None] * self.capacity
        self.terminated = [None] * self.capacity
        
    @property
    def can_sample(self) -> bool:
        """Is able to sample from experience replay."""
        return self.count >= self.sampled_batch_size and self.n_step >= self.training_freq
        
    def add(self, experience: Experience):
        """Add an experience."""
        num_envs = experience.obs.shape[0]
        self.n_step += 1
        
        for i in range(self.num_envs):
            self.recent_idx = (self.recent_idx + 1) % self.capacity
            self.count = min(self.count + 1, self.capacity)
            
            self.obs[self.recent_idx] = experience.obs[i]
            self.action[self.recent_idx] = experience.action[i]
            self.next_obs[self.recent_idx] = experience.next_obs[i]
            self.reward[self.recent_idx] = experience.reward[i]
            self.terminated[self.recent_idx] = experience.terminated[i]
        
    def sample(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        Samples experience batch from it. Default sampling distribution is uniform.

        Returns:
            obs, action, next_obs, reward, terminated
        """
        self.n_step = 0
        # 경험 인덱스 샘플링
        sample_idx = self._sample_idxs()
        
        get_batch_tensor = lambda x: self._get_batch_tensor(x, sample_idx)
        
        obs = get_batch_tensor(self.obs)
        action = get_batch_tensor(self.action)
        next_obs = get_batch_tensor(self.next_obs)
        reward = get_batch_tensor(self.reward)
        terminated = get_batch_tensor(self.terminated)
        
        return obs, action, next_obs, reward, terminated
    
    def _sample_idxs(self) -> np.ndarray:
        """인덱스 랜덤 샘플링."""
        batch_idxs = np.random.randint(self.count, size=self.sampled_batch_size)
        return batch_idxs
    
    def _get_batch_tensor(self, items: list, batch_idx: np.ndarray) -> np.ndarray:
        """`batch_idx`에 따라 리스트 item을 가져옴."""
        return np.array(operator.itemgetter(*batch_idx)(items))

### DQN Configuration

* `training_freq`: 훈련 빈도
* `batch_size`: replay buffer로부터 sampling할 경험 개수
* `capacity`: replay buffer에 저장되는 최대 경험 개수
* `epoch`: 각 훈련 빈도마다 파라미터 업데이트 횟수
* `epsilon_linear_decay`: `(start_t, end_t, start_value, end_value)`
* `gamma`: discount factor

In [16]:
class DQNConfig(NamedTuple):
    training_freq: int
    batch_size: int
    capacity: int
    epoch: int
    epsilon_linear_decay: Tuple[int, int, float, float]
    gamma: float = 0.99

### Epsilon Greedy Policy

Q value로부터 epsilon greedy distribution을 획득한다.  
DQN과 같은 가치 기반 방법에서 사용하며 discrete action에만 사용 가능하다.  
여기서는 discrete action branch가 1이라고 가정한다. 

`epsilon`: 0 ~ 1 사이의 값으로 0에 가까울 수록 exploitation, 1에 가까울 수록 exploration을 수행

In [17]:
from torch.distributions import Categorical

class EpsilonGreedyPolicy:
    def __init__(self, epsilon: float) -> None:
        self.epsilon = epsilon
        
    def sample(self, q_values: torch.Tensor) -> torch.Tensor:
        """
        Q value로부터 action을 샘플링한다.

        Args:
            q_values (torch.Tensor): `(num_envs, num_actions)`의 shape

        Returns:
            torch.Tensor: selected action `(num_envs, 1)`
        """
        num_actions = q_values.shape[1]
        
        # epsilon-greedy probabilities
        greedy_action_prob  = 1.0 - self.epsilon + self.epsilon / num_actions
        non_greedy_action_prob = self.epsilon / num_actions
        
        # get greedy action
        greedy_action = q_values.argmax(dim=1, keepdim=True)
        
        # set epsilon greedy probability distribution
        epsilon_greedy_prob = torch.full_like(q_values, non_greedy_action_prob)
        epsilon_greedy_prob.scatter_(1, greedy_action, greedy_action_prob)
        epsilon_greedy_dist = Categorical(probs=epsilon_greedy_prob)
        return epsilon_greedy_dist.sample().unsqueeze_(-1)

### DQN Agent

In [18]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class DQN:
    def __init__(self,
                 config: DQNConfig,
                 network: nn.Module,
                 optimizer: optim.Optimizer,
                 num_envs: int = 1) -> None:

        self.config = config
        self.network = network
        self.optimizer = optimizer
        self.policy = EpsilonGreedyPolicy(self.config.epsilon_linear_decay[2])
        self.num_envs = num_envs
        
        self.replay_buffer = ReplayBuffer(
            self.config.training_freq,
            self.config.batch_size,
            self.config.capacity,
            self.num_envs
        )
        
        self.global_time_step = 0
        self.training_step = 0
        
        self.num_losses = 0
        self.avg_loss = 0.0
        
    def select_action(self, obs: np.ndarray) -> np.ndarray:
        """
        현재 state에서 action을 선택한다.

        Args:
            obs (np.ndarray): `(num_envs, *obs_shape)`

        Returns:
            np.ndarray: `(num_envs, 1)`
        """
        with torch.no_grad():
            # feed forward
            q_values = self.network(torch.from_numpy(obs))
            
            # action sampling
            action = self.policy.sample(q_values)
        
            return action.detach().cpu().numpy()
        
    def update(self, experience: Experience):
        self.global_time_step += 1
        self.replay_buffer.add(experience)
        
        if self.replay_buffer.can_sample:
            self.train()
            self._epsilon_decay(self.global_time_step)
            
    def train(self):
        for _ in range(self.config.epoch):
            # 경험 샘플링
            exp_batch = self.replay_buffer.sample()
            obs, action, next_obs, reward, terminated = self._to_tensor(exp_batch)
            
            # loss 계산
            td_loss = self.compute_td_loss(obs, action, next_obs, reward, terminated)
            
            # gradient step
            self.optimizer.zero_grad()
            td_loss.backward()
            self.optimizer.step()
            
            # log info
            self.training_step += 1
            self.num_losses += 1
            # incremental mean
            self.avg_loss += 1.0 / self.num_losses * (td_loss.item() - self.avg_loss)
            
    def compute_td_loss(self, 
                        obs: torch.Tensor, 
                        action: torch.Tensor,
                        next_obs: torch.Tensor,
                        reward: torch.Tensor,
                        terminated: torch.Tensor) -> torch.Tensor:
        # 현재 state에서의 모든 action에 대한 Q value 추정
        q_values: torch.Tensor = self.network(obs)
        with torch.no_grad():
            # 다음 state에서의 모든 action에 대한 Q value 추정
            next_q_values = self.network(next_obs)
        
        # 현재 state에서 선택된 action에 대한 Q value
        q_value = q_values.gather(dim=1, index=action)
        # next state에서의 최대 Q value
        next_max_q_value, _ = next_q_values.max(dim=1, keepdim=True)
        # Q target 계산
        not_terminated = 1 - terminated
        q_target = reward + not_terminated * self.config.gamma * next_max_q_value
        
        # TD loss 계산
        td_loss = F.mse_loss(q_target, q_value)
        
        return td_loss
            
    def _to_tensor(self, exp_batch):
        exp_batch_tensor = []
        for item in exp_batch:
            exp_batch_tensor.append(torch.from_numpy(item))
        return tuple(exp_batch_tensor)
    
    def _epsilon_decay(self, t: float):
        start_t, end_t, start_value, end_value = self.config.epsilon_linear_decay
        slope = (end_value - start_value) / (end_t - start_t)
        value = np.clip(slope * (t - start_t) + start_value, end_value, start_value)
        self.policy.epsilon = value
        
    @property
    def log_data(self) -> dict:
        ld = {"Epsilon": (self.policy.epsilon, self.global_time_step)}
        if self.num_losses > 0:
            ld["TD Loss"] = (self.avg_loss, self.training_step)
            self.num_losses = 0
            self.avg_loss = 0.0
        return ld

## Training

### 신경망

아래와 같이 구성됨:

layer 1: 입력층  
layer 2: 은닉층  
layer 3: 출력층 - $Q(S_t, \cdot)$

In [19]:
class CartPoleQNet(nn.Module):
    def __init__(self, num_obs_features: int, num_actions: int) -> None:
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(num_obs_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_actions)
        )
        
    def forward(self, obs: torch.Tensor) -> torch.Tensor:
        return self.layers(obs)

### CartPole 환경 생성

`num_envs`는 `1`

In [20]:
import gym

gym_env = gym.make("CartPole-v1")

#### Observation Space

In [21]:
obs_shape = gym_env.observation_space.shape
obs_shape

(4,)

#### Action Space

In [22]:
num_actions = gym_env.action_space.n
num_actions

2

### DQN Agent 생성

In [23]:
config = DQNConfig(
    training_freq=16,
    batch_size=128,
    capacity=1000,
    epoch=3,
    epsilon_linear_decay=(0, 100000, 0.3, 0.01),
    gamma=0.99
)

In [24]:
network = CartPoleQNet(obs_shape[0], num_actions)
optimizer = optim.Adam(network.parameters(), lr=0.001)

In [25]:
dqn = DQN(
    config,
    network,
    optimizer
)

### Training Start!

In [26]:
from torch.utils.tensorboard import SummaryWriter

logger = SummaryWriter("results/CartPole-v1_DQN")

num_episodes = 501
summary_freq = 10

cumulative_rewards = []

for episode in range(num_episodes):
    cumulative_reward = 0.0
    
    obs, _ = gym_env.reset()
    obs = obs[np.newaxis, ...].astype(np.float32) # (*obs_shape) -> (1, *obs_shape)
    terminated = False
    
    while not terminated:
        # take action and observe
        action = dqn.select_action(obs)
        next_obs, reward, terminated, truncated, _ = gym_env.step(action.item())
        terminated = terminated | truncated
        
        # DQN 업데이트
        experience = Experience(
            obs,
            action,
            next_obs[np.newaxis, ...].astype(np.float32),
            np.array([[reward]]).astype(np.float32),
            np.array([[terminated]], dtype=np.float32).astype(np.float32)
        )
        dqn.update(experience)
        
        # reward 누적
        cumulative_reward += reward
        
        # 현재 observation 업데이트
        obs = experience.next_obs
        
    cumulative_rewards.append(cumulative_reward)
    
    if episode % summary_freq == 0:
        avg_reward = np.mean(cumulative_rewards)
        cumulative_rewards.clear()
        print(f"episode: {episode}, average reward: {avg_reward:.2f}")
        logger.add_scalar("Cumulative Reward", avg_reward, episode)
        for key, value in dqn.log_data.items():
            logger.add_scalar(key, value[0], value[1])
            
logger.flush()
logger.close()

episode: 0, average reward: 12.00
episode: 10, average reward: 12.80
episode: 20, average reward: 11.60
episode: 30, average reward: 10.40
episode: 40, average reward: 10.70
episode: 50, average reward: 12.90
episode: 60, average reward: 10.80
episode: 70, average reward: 11.70
episode: 80, average reward: 18.40
episode: 90, average reward: 14.90
episode: 100, average reward: 23.60
episode: 110, average reward: 30.20
episode: 120, average reward: 46.90
episode: 130, average reward: 84.80
episode: 140, average reward: 120.20
episode: 150, average reward: 91.80
episode: 160, average reward: 51.20
episode: 170, average reward: 69.00
episode: 180, average reward: 349.20
episode: 190, average reward: 270.90
episode: 200, average reward: 112.10
episode: 210, average reward: 174.10
episode: 220, average reward: 360.30
episode: 230, average reward: 419.10
episode: 240, average reward: 359.70
episode: 250, average reward: 453.60
episode: 260, average reward: 429.60
episode: 270, average reward:

아래 커맨드를 anaconda shell에 입력해 결과 확인:

```
$ tensorboard --logdir=results
```